In [ ]:
#This code should only be used in a command line.
#importation des images initiales
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
#%matplotlib inline
import cv2
matplotlib.use("TkAgg")

root_folder = r"E:\Papier\AIAGE\Base"
data_folder = "data_brutes_Perla"
save_folder = "images_labelisees_Perla"


def mypause(interval):
    backend = plt.rcParams['backend']
    if backend in matplotlib.rcsetup.interactive_bk:
        figManager = matplotlib._pylab_helpers.Gcf.get_active()
        if figManager is not None:
            canvas = figManager.canvas
            if canvas.figure.stale:
                canvas.draw()
            canvas.start_event_loop(interval)
            return

for folder_name in os.listdir(os.path.join(root_folder, data_folder)):
    folder_path = os.path.join(root_folder, data_folder, folder_name)

    # Liste des fichiers d'images
    list_files_orig = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    for path_np in tqdm(list_files_orig):
        #path_to_save_labeled_image.
        save_image_path = os.path.join(root_folder, save_folder, folder_name)  # Update with your desired save path
        os.makedirs(save_image_path, exist_ok=True)
        path_ = os.path.basename(os.path.normpath(path_np))        
        if os.path.exists(os.path.join(save_image_path, path_)):
            pass
        else :
            npzfile = np.load(path_np)
            image = npzfile['image']

            print('Select an option:')
            options = ['0.  Normal', '1.  Plomb','2.  Appareil','3.  fil',
                       '4.  implant','5.  Appreil+plomb','6.  fil+plomb','7.  Matériel ',
                       '8.  Matériel+plomb','9.  Matériel+appareil','10. Matériel+fil',
                       '11. Matériel+appareil+plomb','12. Matériel+fil+plomb','13. Autre']


            for option in options:
                print(option)

            plt.imshow(image,cmap='gray')
            plt.show(block=False)
            #plt.pause(0.001)
            mypause(1)

            condition = True

            while condition:

                try:
                    label_tmp = int(input())
                    if label_tmp in range(0, len(options)):
                        condition = False
                        break  # Exit the loop if the input is a valid integer within the range of options
                    else:
                        print('Invalid choice. Please enter an integer from the list.')
                except ValueError:
                    print('Invalid input. Please enter an integer from the list.')

            # Save labeled image as npz file
            np.savez(os.path.join(save_image_path, path_), image=npzfile['image'], ID_ANONp=npzfile['ID_ANONp'],
                     ID_ANONs=npzfile['ID_ANONs'], ID_ANONi=npzfile['ID_ANONi'],
                     Sexe=npzfile['Sexe'], Age=npzfile['Age'], Label=label_tmp,
                     slope=npzfile['slope'], intercept=npzfile['intercept'])
            npzfile.close()

print('-------------------------------------fin--------------------------------------------')




In [ ]:
#######recodage#####
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import shutil
import math
import time
path_to_scan = r"E:\Papier\AIAGE\Base\triage_label"
#path_to_scan = r"E:\Papier\AIAGE\Base\test3"
new_destination=r"E:\Papier\AIAGE\Base\image_corrigees"
for label_folder in os.listdir(path_to_scan):
    if not os.path.isdir(os.path.join(path_to_scan, label_folder)):
        continue

    print(f"************************Traitement du dossier {label_folder}**********************")

    path_to_folder = os.path.join(path_to_scan, label_folder)
    list_files_orig = [os.path.join(path_to_folder, f) for f in os.listdir(path_to_folder) if os.path.isfile(os.path.join(path_to_folder, f))]

    images = []
    ages = [] # Modifié pour avoir une convention de nommage avec des minuscules
    labels = [] # Modifié pour avoir une convention de nommage avec des minuscules

    for i in tqdm(range(0, len(list_files_orig), 4)):
        id = 1
        current_images = []
        npzfiles = []

        for j in range(4):
            if i+j < len(list_files_orig) and os.path.isfile(list_files_orig[i+j]) :
                npzfile = np.load(list_files_orig[i+j])
                npzfiles.append(npzfile)
                image = npzfile['image']
                label = npzfile['Label'] 
                labels.append(label)
                age = npzfile['Age'] 
                ages.append(age.tolist())
                current_images.append(image)
               
                
        fig, subplots = plt.subplots(2, 2, figsize=(11.69, 8.27))
        z = 0
        for k in range(0, len(current_images)):
            ax = subplots[z][k%2]
            ax.set_title(f"ID {id}, Age{age}")
            ax.imshow(current_images[k], cmap='gray')
            id = id+1
            if k>0 :
                if k%2 == 1:
                    z = z+1
        plt.show()
        condition = True
        corrige=False
        nbr_lot = len(current_images)
        
        while condition:
            try:

                img_id = int(input("Entrez l'ID de l'image à modifier (0 pour passer à la série suivante)"))
                if img_id in range(0, nbr_lot+1):
                    if img_id == 0 :
                        condition = False
                        
                        for j,image in enumerate(current_images):
                            old_path = list_files_orig[i+j]
                            old_Label=labels[i+j]
                            print("old path",old_path+"")
                            new_path = os.path.join(new_destination,f"label_{old_Label}",os.path.basename(old_path))
                            print("new path",new_path+"")
                            shutil.copy(old_path,new_path)
                        break
                    else :
                        new_label = int(input("Entrez le nouveau label : "))
                        img_index = i + img_id - 1
#                         labels[img_index] = new_label
                        
                        if new_label== label :
                           print("in condition to copy")
                           old_path = list_files_orig[img_index]
                           print("old path",old_path+"")
                           new_path = os.path.join(new_destination,f"label_{ label}",os.path.basename(old_path))
                           #print("new path",new_path+"")
                           shutil.copy(old_path,new_path)
                        else:
                            labels[img_index] = new_label
                            old_path = list_files_orig[img_index]
                            new_path = os.path.join(new_destination,f"label_{ new_label}",os.path.basename(old_path))
                            np.savez(old_path, image=npzfiles[img_id-1]['image'], ID_ANONp=npzfiles[img_id-1]['ID_ANONp'],
                                 ID_ANONs=npzfiles[img_id-1]['ID_ANONs'], ID_ANONi=npzfiles[img_id-1]['ID_ANONi'],
                                 Sexe=npzfiles[img_id-1]['Sexe'], Age=npzfiles[img_id-1]['Age'], Label=new_label,
                                 slope=npzfiles[img_id-1]['slope'], intercept=npzfiles[img_id-1]['intercept'])
                            npzfiles[img_id-1].close()
                            shutil.copy(old_path,new_path)
#                             npzfile.close()
#                             plt.close("all")

                       
            except ValueError:
                print("Entrée invalide, veuillez entrer un entier valide.") 

In [ ]:
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import shutil
import math
import time
#path_to_scan = r"E:\Papier\AIAGE\Base\triage_label"
#path_to_scan = r"E:\Papier\AIAGE\Base\image_test"
path_to_scan=r"E:\Papier\AIAGE\Base\image_corrigees"
# new_destination=r"E:\Papier\AIAGE\Base\test3"
for label_folder in os.listdir(path_to_scan):
    if not os.path.isdir(os.path.join(path_to_scan, label_folder)):
        continue

    print(f"************************Traitement du dossier {label_folder}**********************")

    path_to_folder = os.path.join(path_to_scan, label_folder)
    list_files_orig = [os.path.join(path_to_folder, f) for f in os.listdir(path_to_folder) if os.path.isfile(os.path.join(path_to_folder, f))]

    images = []
    ages = [] # Modifié pour avoir une convention de nommage avec des minuscules
    labels = [] # Modifié pour avoir une convention de nommage avec des minuscules

    for i in tqdm(range(0, len(list_files_orig), 4)):
        id = 1
        current_images = []
        npzfiles = []

        for j in range(4):
            if i+j < len(list_files_orig) and os.path.isfile(list_files_orig[i+j]) :
                npzfile = np.load(list_files_orig[i+j])
                npzfiles.append(npzfile)
                image = npzfile['image']
                label = npzfile['Label'] 
                labels.append(label)
                age = npzfile['Age'] 
                ages.append(age.tolist())
                current_images.append(image)
               
                
                       
        fig, subplots = plt.subplots(2, 2, figsize=(11.69, 8.27))
        z = 0
        for k in range(0, len(current_images)):
            ax = subplots[z][k%2]
            ax.set_title(f"ID {id}, Age{age}")
            ax.imshow(current_images[k], cmap='gray')
            id = id+1
            if k>0 :
                if k%2 == 1:
                    z = z+1
        plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageOps 
import cv2 
import numpy as np 
from tqdm import tqdm

# Define the path to scan
path_to_scan = r"E:\Papier\AIAGE\Base\image_corrigees\label_4"

# Get a list of files to load
list_files_orig = [os.path.join(path_to_scan, f) for f in os.listdir(path_to_scan) if os.path.isfile(os.path.join(path_to_scan, f))]
for path_np in list_files_orig :
    npzfile = np.load(path_np)
    image = npzfile['image']
    Sexe = npzfile['Sexe']
    Age = npzfile['Age']
    Label = npzfile['Label']
    plt.imshow(image,cmap='gray')
    plt.title(f'Age:{Age},label:{Label}'.format(Age,Label))
    plt.show()